`IEnumerable<TSource>`→`scalar`  
  
| Method    | Description | SQL equivalents |
| -------- | ------- | ------- |
|`Count`, <br>`LongCount` |Returns the number of elements in the input sequence,<br> optionally satisfying a predicate |COUNT (...)
|`Min`, `Max`|Returns the smallest or largest element in the sequence|MIN (...), <br> MAX (...)
|`Sum`, `Average`|Calculates a numeric sum or average over elements in the <br> sequence| SUM (...), <br> AVG (...)
|`Aggregate`| Performs a custom aggregation | Exception thrown

### Count and LongCount

|Argument | Type
| -----| ----
|***Source sequence*** |`IEnumerable<TSource>`
|***Predicate*** (optional) |`TSource => bool`

In [ ]:
//Count simply enumerates over a sequence, returning the number of items:
int fullCount = new int[] { 5, 6, 7 }.Count(); // 3

The `internal implementation` of ***Enumerable.Count*** tests the input sequence to see whether it happens to `implement ICollection<T>`. If it does, it simply calls `ICollection<T>.Count`; otherwise, it `enumerates over every item`, `incrementing a counter`

In [ ]:
//You can optionally supply a predicate:

int digitCount = "pa55w0rd".Count (c => char.IsDigit (c)); // 3

***LongCount*** does the `same job` as ***Count*** but returns a `64-bit integer`, allowing for sequences of `greater than` ***two billion elements***.

### Min and Max

|Argument | Type
| -----| ----
|***Source sequence*** |`IEnumerable<TSource>`
|***Result selector*** (optional) |`TSource => TResult`

In [ ]:
//Min and Max return the smallest or largest element from a sequence:

int[] numbers = { 28, 32, 14 };
int smallest = numbers.Min(); // 14;
int largest = numbers.Max(); // 32;

In [ ]:
int[] numbers = { 28, 32, 14 };

//If you include a selector expression, each element is first projected:
int smallest = numbers.Max (n => n % 10); // 8;

A ***selector expression*** is `mandatory(اجباری)` if the items themselves are not intrinsically `comparable`—in other words, if they do not implement `IComparable<T>`

In [ ]:
Purchase runtimeError = dbContext.Purchases.Min (); // Error
decimal? lowestPrice = dbContext.Purchases.Min (p => p.Price); // OK

A ***selector expression*** determines `not only` how elements are `compared`, but also the `final result`.

In [ ]:
//To get the cheapest purchase, you need a subquery
Purchase cheapest = dbContext.Purchases
.Where (p => p.Price == dbContext.Purchases.Min (p2 => p2.Price))
.FirstOrDefault();

//best paractice is use OrderBy for this solution

### Sum and Average

|Argument | Type
| -----| ----
|***Source sequence*** |`IEnumerable<TSource>`
|***Result selector*** (optional) |`TSource => TResult`

In [ ]:
decimal[] numbers = { 3, 4, 8 };
decimal sumTotal = numbers.Sum(); // 15
decimal average = numbers.Average(); // 5 (mean value)

In [ ]:
int combinedLength = names.Sum (s => s.Length); // 19

Average Result Types  
  
|Selector type |Result type
|---- | ----
|**decimal** |`decimal`
|**float** |`float`
|**int**, **long**, **double** |`double`

In [ ]:
//This means that the following does not compile (“cannot convert double to int”):
int avg = new int[] { 3, 4 }.Average();

//But this will compile:
double avg1 = new int[] { 3, 4 }.Average(); // 3.5


### Aggregate

***Aggregate*** allows you to specify a `custom accumulation(جمع آوری) algorithm` for implementing `unusual aggregations`.

In [ ]:
int sum = numbers.Aggregate (0, (total, n) => total + n); // 6

//The first argument to Aggregate is the seed,from which accumulation starts
// اولین مقدار را صفر در نظر میگیرد، و دومین مقدار 1 میشود

#### Unseeded aggregations

In [ ]:
int[] numbers = { 1, 2, 3 };

//we were calculating 0 + 1 + 2 + 3;
int sum = numbers.Aggregate (0, (total, n) => total + n); // 6

//now we’re calculating 1 + 2 + 3.
int sum1 = numbers.Aggregate ((total, n) => total + n); // 6

In [ ]:
int[] numbers = { 1, 2, 3 };
int x = numbers.Aggregate (0, (prod, n) => prod * n); // 0*1*2*3 = 0
int y = numbers.Aggregate ( (prod, n) => prod * n); // 1*2*3 = 6

***unseeded aggregations*** have the advantage of being `parallelizable` without requiring the use of special overloads.